# Regression Model in Keras - Mauro Valls

What you will see:
1. Functions for simplifying processes
2. Loading the dataset
3. Validating the dataset
4. Splitting the dataset into training and testing sets
5. Model A - Baseline results
6. Normalizing the input features
7. Model B - Results after normalization
8. Model C - Results with increased epochs
9. Model D - Results with three hidden layers
10. Final comparative results

In [ ]:
import numpy as np
import pandas as pd
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Functions for simplifying processes

In [ ]:
def load_data(file_path):
    """Loads data from a CSV file or from the URL"""
    try:
      # Checks if the file exists locally
        if os.path.isfile(file_path):
            print(f"Loading data from the file'{file_path}'")
            data = pd.read_csv(file_path, header=0)
        else:
            # Load data from the URL if the file is not found locally
            print(f"File not found: {file_path}. Loading from the URL.")
            url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
            data = pd.read_csv(url)
        return data
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

def mean(scores):
    """Calculate the average value from a list of scores"""
    # Checks if the list is not empty
    if len(scores) > 0:
        # Calculates the mean and round it to 2 decimal places
        return round(np.mean(scores), 2)
    return None

def standard_deviation(scores):
    """Calculate the standard deviation of a list of scores"""

    # Checks if the list is not empty
    if scores:
         # Calculates the standard deviation and round it to 2 decimal places
        return round(np.std(scores), 2)
    return None

def report(experiment_label, avg_error_value, deviation_value):
    """Generate a summary DataFrame with experiment results."""

    # Creates a dictionary to summarize the results
    result_summary = {
        'Experiment': [experiment_label],
        'Average Error (Mean MSE)': [avg_error_value],
        'Error Deviation (Deviation MSE)': [deviation_value]
    }

    # Converts the dictionary into a pandas DataFrame
    return pd.DataFrame(result_summary)

def model_a(inputs=3):
    """Creates a neural network model with:
    - One hidden layer of 10 nodes with ReLU activation.
    - Adam optimizer and mean squared error as the loss function.
    """

    # Initialize the model (Sequential as layers are added in a linear stack)
    model = Sequential()

    # Add hidden layer with 10 nodes
    model.add(Dense(10, activation="relu", input_shape=(inputs,)))
    # Add the output layer with 1 node
    model.add(Dense(1))
    # Compiles the model with Adam optimizer and mean squared error as the loss
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def mean_squared_root_evaluation(trained_model, inputs, target, num_epochs=50, display_output=1):
    # Splits the data into training (70%) and testing (30%) sets
    X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.3, random_state=24)
    print(f"Training set size: {X_train.shape}, {y_train.shape}")
    print(f"Testing set size: {X_test.shape}, {y_test.shape}")

    # Trains the model using the training data
    trained_model.fit(X_train, y_train, epochs=num_epochs, verbose=display_output)

    # Uses the trained model to predict on the test data
    predictions = trained_model.predict(X_test)

    # Calculates the mean squared error between predictions and actual values
    error = mean_squared_error(y_test, predictions)
    return error

def calculate_error_stats(inputs, target_values, model, iterations=50, epochs=50, verbose=0):
    """Calculate the average and standard deviation of prediction errors"""
    error_list = []
    for round_number in range(iterations):
        # Record the start time of the iteration
        start_time = time.time()
        print("-" * 40)
        print(f"Starting iteration: {round_number + 1}")

        # Evaluate the prediction error for the current iteration
        current_error = mean_squared_root_evaluation(
            trained_model=model,
            inputs=inputs,
            target=target_values,
            num_epochs=epochs,
            display_output=verbose
        )
        # Appends the error to the list
        error_list.append(current_error)
        print(f"Iteration finished in {time.time() - start_time:.2f} seconds.")

    print(f"Completed {iterations} iterations. Error list: {error_list}")

    # Calculates the average error
    average_error = mean(error_list)
    # Calculate the standard deviation of the errors
    error_std_dev = standard_deviation(error_list)

    print("-" * 70)
    print(f"Average Error: {average_error}, Standard Deviation: {error_std_dev}")

    return average_error, error_std_dev

def model_d(inputs=3):
    """Create a neural network model with:

    - Three hidden layers, each with 10 nodes using ReLU activation.
    - Adam optimizer and mean squared error as the loss function.
    """

    # Initialize the neural network
    model = Sequential()

    # Adds hidden layers with ReLU activation and 10 nodes
    model.add(Dense(10, activation="relu", input_shape=(inputs,)))
    model.add(Dense(10, activation="relu"))
    model.add(Dense(10, activation="relu"))

    # Adds output layer
    model.add(Dense(1))

    # Compiles the model with the 'adam' optimizer and loss function (MSE)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

## Loading the dataset

If the file path exists, we will load its contents into the DataFrame.

In [ ]:
file_path = "/concrete_data.csv" # This can depend on where you saved the csv file. **Change might be needed **
# Loads the data into a DataFrame
df = load_data(file_path)

Loading data from the file'/concrete_data.csv'


## Validating the dataset

In [ ]:
df.columns # We check the columns of the df

Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
      dtype='object')

In [ ]:
df.describe() # key statistical measures (Count, Mean, etc...)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [ ]:
df.info() # Provides a concise summary of the DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
 8   Strength            1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


We will be splitting the data into training and testing sets, holding 30% of the data for testing.

In [ ]:
print(f"DataFrame dimensions: Rows={df.shape[0]}, Columns={df.shape[1]}") # Displayes the shape of the DataFrame

DataFrame dimensions: Rows=1030, Columns=9


In [ ]:
# Now we will check if there is any data missing
df.isnull().sum()

,0
Cement,0
Blast Furnace Slag,0
Fly Ash,0
Water,0
Superplasticizer,0
Coarse Aggregate,0
Fine Aggregate,0
Age,0
Strength,0


As we can see, the data is all loaded and good to go. Now we will normalize the data to ensure that all the features have the same data scale. This is important to improve the performance and stability of the machine learning models.

## Splitting the dataset into training and testing sets

In [ ]:
# Here we show all the columns of the df
columns = df.columns
print(columns)

Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
      dtype='object')


### We will now split the data into predictors and targets

In [ ]:
# Here we extract the predictors, excluding the 'Strength' column, which will be the target
strength_column = "Strength"
column_predictors = [col for col in columns if col != strength_column]
predictor_df = df[column_predictors]
predictor_df.head(4) # We show the first 4 rows of the predictor's df

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365


In [ ]:
# Here we will select the 'Strength' column to be our target df
target_df = df[[strength_column]]
target_df.head(4) # We show the first 4 rows of the target's df

,Strength
0,79.99
1,61.89
2,40.27
3,41.05


# A. Build a baseline model

### Requirements:

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the
train_test_split
helper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

Submit your Jupyter Notebook with your code and comments.

### Here we build and train the model

In [ ]:
compiled_model = model_a(inputs=len(column_predictors))
print(len(column_predictors))

8


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
iterations = 50
epochs = 50
verbose = 0

# We compile the model...
average_mean_squared_error_A, error_deviation_A = calculate_error_stats(
    inputs=predictor_df,
    target_values=target_df,
    model=compiled_model,
    iterations=iterations,
    epochs=epochs,
    verbose=verbose
)

----------------------------------------
Starting iteration: 1
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration finished in 3.76 seconds.
----------------------------------------
Starting iteration: 2
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Iteration finished in 2.94 seconds.
----------------------------------------
Starting iteration: 3
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 4.35 seconds.
----------------------------------------
Starting iteration: 4
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 2.92 seconds.
----------------------------------------
Starting iteration: 5
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 

## Report for model A

In [ ]:
experiment = "A) - Baseline Raw model (50 epochs)"
results_a_df = report(experiment, average_mean_squared_error_A, error_deviation_A)
results_a_df

,Experiment,Average Error (Mean MSE),Error Deviation (Deviation MSE)
0,A) - Baseline Raw model (50 epochs),74.22,63.77


# B. Normalize the data
Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?

### Data before normalizing

In [ ]:
predictor_df.head(4) # We show the first 4 rows of the predictor's df

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365


### Now we will normalize the input data
We normalize the data by subtracting the mean from each predictor and dividing the result by its standard deviation.

In [ ]:
predictor_df_normalized = (predictor_df - predictor_df.mean()) / predictor_df.std()
predictor_df_normalized.head(4) # We show the first 4 rows of the normalized predictor's df

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221


In [ ]:
iterations = 50
epochs = 50
verbose = 0


compiled_model = model_a(inputs=len(column_predictors))

# We compile the model...
average_mean_squared_error_B, error_deviation_B = calculate_error_stats(
    inputs=predictor_df_normalized,
    target_values=target_df,
    model=compiled_model,
    iterations=iterations,
    epochs=epochs,
    verbose=verbose
)

----------------------------------------
Starting iteration: 1
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration finished in 4.90 seconds.
----------------------------------------
Starting iteration: 2
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 2.90 seconds.
----------------------------------------
Starting iteration: 3
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 2.87 seconds.
----------------------------------------
Starting iteration: 4
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 3.95 seconds.
----------------------------------------
Starting iteration: 5
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 3.16 seconds.
----------------------------------------
Starting iteration: 6
Train

## Report for model B

In [ ]:
experiment = "B) Normalized model (50 epochs)"
results_b_df = report(experiment, average_mean_squared_error_B, error_deviation_B)
results_b_df

,Experiment,Average Error (Mean MSE),Error Deviation (Deviation MSE)
0,B) Normalized model (50 epochs),50.18,40.0


# C. Increate the number of epochs
Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

In [ ]:
iterations = 50
epochs = 100
verbose = 0


compiled_model = model_a(inputs=len(column_predictors))

# We compile the model...
average_mean_squared_error_c, error_deviation_c = calculate_error_stats(
    inputs=predictor_df_normalized,
    target_values=target_df,
    model=compiled_model,
    iterations=iterations,
    epochs=epochs,
    verbose=verbose
)

----------------------------------------
Starting iteration: 1
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Iteration finished in 6.98 seconds.
----------------------------------------
Starting iteration: 2
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Iteration finished in 6.56 seconds.
----------------------------------------
Starting iteration: 3
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Iteration finished in 6.65 seconds.
----------------------------------------
Starting iteration: 4
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 6.13 seconds.
----------------------------------------
Starting iteration: 5
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 6.06 seconds.
----------------------------------------
Starting iteration: 6
Traini

## Report for model C

In [ ]:
experiment = "C) Normalized model (100 epochs)"
results_c_df = report(experiment, average_mean_squared_error_c, error_deviation_c)
results_c_df

,Experiment,Average Error (Mean MSE),Error Deviation (Deviation MSE)
0,C) Normalized model (100 epochs),42.24,15.91


# D. Increase the number of hidden layers
Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

How does the mean of the mean squared errors compare to that from Step B?

In [ ]:
iterations = 50
epochs = 50
verbose = 0


compiled_model = model_d(inputs=len(column_predictors))

# We compile the model...
average_mean_squared_error_d, error_deviation_d = calculate_error_stats(
    inputs=predictor_df_normalized,
    target_values=target_df,
    model=compiled_model,
    iterations=iterations,
    epochs=epochs,
    verbose=verbose
)

----------------------------------------
Starting iteration: 1
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Iteration finished in 4.40 seconds.
----------------------------------------
Starting iteration: 2
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 4.55 seconds.
----------------------------------------
Starting iteration: 3
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 2.98 seconds.
----------------------------------------
Starting iteration: 4
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 3.13 seconds.
----------------------------------------
Starting iteration: 5
Training set size: (721, 8), (721, 1)
Testing set size: (309, 8), (309, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Iteration finished in 3.94 seconds.
----------------------------------------
Starting iteration: 6
Traini

## Report for model D

In [ ]:
experiment = "D) Normalized model 3 hidden layers (50 epochs)"
results_d_df = report(experiment, average_mean_squared_error_d, error_deviation_d)
results_d_df

,Experiment,Average Error (Mean MSE),Error Deviation (Deviation MSE)
0,D) Normalized model 3 hidden layers (50 epochs),40.13,19.46


# Final results

In [ ]:
final_results_df = pd.concat([results_a_df, results_b_df, results_c_df, results_d_df], ignore_index=True)

# We Display the combined DataFrame
final_results_df

,Experiment,Average Error (Mean MSE),Error Deviation (Deviation MSE)
0,A) - Baseline Raw model (50 epochs),74.22,63.77
1,B) Normalized model (50 epochs),50.18,40.00
2,C) Normalized model (100 epochs),42.24,15.91
3,D) Normalized model 3 hidden layers (50 epochs),40.13,19.46


The mean squared error (MSE) is an essential metric used to evaluate how well a regression model fits the testing data. A lower MSE means the model predicts more accurately. The standard deviation (Deviation MSE) reflects the consistency of the model's predictions.

**1. Model A - Baseline Raw Model (50 epochs):**

- This model has the highest mean MSE (74.22) and a very large deviation (63.77). This indicates that without normalization, the model struggles with both accuracy and consistency.

**2. Model B - Normalized Model (50 epochs):**
- After applying normalization, the mean MSE significantly drops to 50.18, with a reduced error deviation of 40.00. Normalizing the data improves the model's performance by making it more accurate and consistent.

**3. Model C - Normalized Model (100 epochs):**
- With the same single hidden layer but double the epochs (100), the mean MSE decreases further to 42.24, and the deviation reduces to 15.91. This demonstrates that increasing the number of epochs allows the model to learn better, resulting in higher accuracy and more consistent predictions with a lower error deviation.

**4. Model D - Normalized Model with 3 Hidden Layers (50 epochs):**
- This model achieves the best performance, with the lowest mean MSE (40.13) and a small error deviation (19.46). Using three hidden layers improves the model's ability to capture complex patterns in the data, even with only 50 epochs.


**Conclusion:**
Experimenting with techniques such as normalizing the input data, increasing the number of epochs to balance accuracy and training time, and adding more hidden layers to improve the model's capacity to capture complex relationships in the data can help achieve the best results.